Deploy interativo

In [ ]:
import os
import mlflow.pyfunc
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
import mlflow.tracking
import pandas as pd # Para simular a entrada do Streamlit

print("Iniciando Notebook para Teste de Churn Prediction com Streamlit/MLflow")

try:
    spark = SparkSession.builder \
        .appName("NotebookSparkModelLoader") \
        .config("spark.local.dir", os.path.join(os.getcwd(), "spark_notebook_tmp")) \
        .config("spark.sql.warehouse.dir", os.path.join(os.getcwd(), "spark-warehouse-notebook")) \
        .getOrCreate()
    print("SparkSession iniciada com sucesso.")
except Exception as e:
    print(f"Erro ao iniciar SparkSession: {e}")

    spark = SparkSession.builder.getOrCreate()
    print("Usando SparkSession existente.")


# --- Função para carregar o Modelo MLflow ---
def load_mlflow_churn_model_notebook():
    try:

        mlflow_db_path = "C:/MONICA/Estudos/mlf_data/mlflow.db" 
        tracking_uri = f"sqlite:///{mlflow_db_path}" 
        mlflow.set_tracking_uri(tracking_uri)
        
        print(f"MLflow Tracking URI configurado para: {tracking_uri}") # Mensagem de depuração
        
        client = mlflow.tracking.MlflowClient()

        experiment = client.get_experiment_by_name("churn_prediction") 
        
        if experiment:
            runs = client.search_runs(
                experiment_ids=[experiment.experiment_id],
                order_by=["start_time DESC"],
                max_results=1
            )
            if runs:
                latest_run = runs[0]
                
                model_uri = f"runs:/{latest_run.info.run_id}/churn_model" 
                
                print(f"Tentando carregar modelo da URI: {model_uri}")
                return mlflow.pyfunc.load_model(model_uri)
            else:
                print("Nenhuma execução (run) encontrada para o experimento 'churn_prediction'.")
                return None
        else:
            print("Experimento 'churn_prediction' não encontrado no MLflow Tracking Server.")
            return None
    except Exception as e:
        print(f"Erro ao carregar o modelo MLflow: {e}. Verifique o caminho e as permissões.")
        return None

# Carrega o modelo
model_mlflow = load_mlflow_churn_model_notebook()

if model_mlflow:
    print("Modelo MLflow carregado com sucesso!")

    print("\nSimulando uma previsão com dados de exemplo:")
    
    test_data = [(
        "No", # Valor dummy para a coluna 'Churn'
        100, "No", 0, 200.0, 100, 150.0, 90, 180.0, 80, 10.0, 5, 2
    )]

    input_schema = StructType([
        StructField("Churn", StringType(), True),
        StructField("Account length", IntegerType(), True),
        StructField("International plan", StringType(), True),
        StructField("Number vmail messages", IntegerType(), True),
        StructField("Total day minutes", DoubleType(), True),
        StructField("Total day calls", IntegerType(), True),
        StructField("Total eve minutes", DoubleType(), True),
        StructField("Total eve calls", IntegerType(), True),
        StructField("Total night minutes", DoubleType(), True),
        StructField("Total night calls", IntegerType(), True),
        StructField("Total intl minutes", DoubleType(), True),
        StructField("Total intl calls", IntegerType(), True),
        StructField("Customer service calls", IntegerType(), True)
    ])
    
    input_df_spark = spark.createDataFrame(test_data, schema=input_schema)
    
    input_pandas_df = input_df_spark.toPandas()
    
    predictions = model_mlflow.predict(input_pandas_df)
    
    predicted_label_index = predictions[0]
    predicted_churn = "Sim (Churn)" if predicted_label_index == 1.0 else "Não (Sem Churn)"
    
    print(f"Resultado da Previsão: {predicted_churn}")
    print("\nTeste concluído no notebook.")
else:
    print("Não foi possível carregar o modelo. Verifique as mensagens de erro acima.")


2025/07/14 14:03:56 INFO mlflow.spark: File 'C:\Users\mosme\AppData\Local\Temp\tmpao40qydf\churn_model\sparkml' is already on DFS, copy is not necessary.


Iniciando Notebook para Teste de Churn Prediction com Streamlit/MLflow
SparkSession iniciada com sucesso.
MLflow Tracking URI configurado para: sqlite:///C:/MONICA/Estudos/mlf_data/mlflow.db
Tentando carregar modelo da URI: runs:/08622310dc5043d2b3f491048ba46df6/churn_model
Modelo MLflow carregado com sucesso!

Simulando uma previsão com dados de exemplo:
Resultado da Previsão: Não (Sem Churn)

Teste concluído no notebook.
